In [1]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks
import re

# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

from dotenv import load_dotenv
import os
load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
# Load and display the data from CSV to confirm
tsla_df = pd.read_csv('TSLA_stock_price.csv')
print(tsla_df.head())    

         date  1. open  2. high  3. low  4. close   5. volume ticker
0  2024-05-08   171.59   176.06  170.15    174.72  79969488.0   TSLA
1  2024-05-07   182.40   183.26  177.40    177.81  75045854.0   TSLA
2  2024-05-06   183.80   187.56  182.20    184.76  84390253.0   TSLA
3  2024-05-03   182.10   184.78  178.42    181.19  75491539.0   TSLA
4  2024-05-02   182.86   184.60  176.02    180.01  89148041.0   TSLA


In [3]:
#Defining a function to clean the column names
def clean_column_name(name):
    # Remove all non-letter characters
    cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    return cleaned_name

In [4]:
tsla_df

,date,1. open,2. high,3. low,4. close,5. volume,ticker
0,2024-05-08,171.59,176.0600,170.15,174.72,79969488.0,TSLA
1,2024-05-07,182.40,183.2600,177.40,177.81,75045854.0,TSLA
2,2024-05-06,183.80,187.5600,182.20,184.76,84390253.0,TSLA
3,2024-05-03,182.10,184.7800,178.42,181.19,75491539.0,TSLA
4,2024-05-02,182.86,184.6000,176.02,180.01,89148041.0,TSLA
...,...,...,...,...,...,...,...
3484,2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,TSLA
3485,2010-07-02,23.00,23.1000,18.71,19.20,5139800.0,TSLA
3486,2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,TSLA
3487,2010-06-30,25.79,30.4192,23.30,23.83,17187100.0,TSLA


In [5]:
# Cleaning up column names for 'tsla_df'
tsla_df.columns = [clean_column_name(col) for col in tsla_df.columns]
print(tsla_df.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')


In [6]:
# Converting the "date" column to timestamp
tsla_df['date'] = pd.to_datetime(tsla_df['date'])

In [7]:
# Defining the stocks feature group
tesla_fg = fs.get_or_create_feature_group(
    name="tesla_stock",
    description="Tesla stock dataset from alpha vantage",
    version=3,
    primary_key=["ticker"],
    event_time=['date'],
    online_enabled=False,
)

2024-05-09 13:46:25,296 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [8]:
#Inserting the stock data into the stocks feature group
tesla_fg.insert(tsla_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/797023


Uploading Dataframe: 0.00% |          | Rows 0/3489 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tesla_stock_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stock_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1eb9c874450>, None)

In [9]:
#Collecting news df
news_df = pd.read_csv('news_articles_ema.csv')

In [10]:
#Dropping exp mean 7 days
news_df_updated = news_df.drop(columns=['exp_mean_7_days'])

In [11]:
#Updating date to datetime
news_df_updated['date'] = pd.to_datetime(news_df_updated['date'])

In [12]:
#Defining the news feature group
news_sentiment_fg = fs.get_or_create_feature_group(
    name='news_sentiment_updated',
    description='News sentiment from Polygon',
    version=3,
    primary_key=['ticker'],
    event_time=['date'],
    online_enabled=False,
)

2024-05-09 13:55:13,441 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [13]:
#Inserting the news data into the news feature group
news_sentiment_fg.insert(news_df_updated)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/798028


Uploading Dataframe: 0.00% |          | Rows 0/75 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/news_sentiment_updated_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1eb9c877fd0>, None)